In [5]:
%%writefile "./py-files/bayesian_ts_watch_opponent_stick_with_winner_watch_oppo.py"
import json
import numpy as np
import pandas as pd

bandit_state = None 
total_reward = 0
last_step_bandit = None
opponent_last = None
    
def bayesian_ts_bandit(observation, configuration):
    global bandit_state,total_reward,last_step_bandit, opponent_last
    best_agent = None
    if observation.step%100 == 0:
        #print(f"Observations {observation}")
        print(f"3 step strategy watch oppo {total_reward}")
        
    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(100)]
    else:       
        reward = observation.reward - total_reward
        total_reward += reward
        opponent_current = observation.lastActions[1]
        # updating bandit_state using the result of the previous step       
        if reward > 0:
            # If winner increase the wins for that particular bandit
            bandit_state[last_step_bandit][0] += reward 
            best_agent = last_step_bandit
        
        # Increase the trials - wins for that particular bandit
        #bandit_state[last_step_bandit][1] += 1 - reward
        else:
            opponent_current = observation.lastActions[1]
            if opponent_last:
                if opponent_last == opponent_current:
                    bandit_state[opponent_last][0] += 1
                    best_agent = opponent_last
                else:
                    bandit_state[opponent_last][1] += 1

        opponent_last = opponent_current  
        bandit_state[last_step_bandit][1] += 1 - reward
        
    best_proba = -1
    if best_agent is None:
        for k in range(100):
            proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
            if proba > best_proba:
                best_proba = proba
                best_agent = k

    last_step_bandit = best_agent

    return best_agent

Overwriting ./py-files/bayesian_ts_watch_opponent_stick_with_winner_watch_oppo.py
